In [67]:
#@author: 高良科
#编辑日期：2021//12-4
import pandas as pd
import lightgbm as lgb
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score, fbeta_score, precision_score, recall_score, roc_auc_score
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sklearn.naive_bayes import GaussianNB  
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
import joblib
import time
import jieba
import numpy as np
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings("ignore")

### s输入数据字符串列表，每个字符串最好是标题+正文

### 文章越长越好，把标题和正文组合在一起输进去最好。文字太短的预测不是很准

In [68]:
# 示例数据
sample=['黄健翔拍时装大片　承认口无遮拦 ２００６年之前，他只是最好的体育节目主持人之一。２００４年之后，他在人们眼中忽然多元起来。仿佛从楚门的世界中完美谢幕，进入了一个由自我掌控的更真实、也更冒险的环境中。不管是无心插柳还是有心栽花，我们都依稀从他身上看到，跳跃的自由在荆棘中骄傲地微笑。可是让他在公益、文化、娱乐中选一个来界定自己，他还是一如既往写下了——体育节目主持人。一身运动装的黄健翔带着一身阳光柳絮走进摄影棚，再彬彬有礼也让人先入为主地存了活泼的印象，何况他正活灵活现地形容着刚刚在路上看到的天津包子铺。大家热络地聊了几句，他的话题还是满怀憧憬地往这上面牵引。结果是助理数了数人头出发买包子去了。一大桌子人热热闹闹吃完开工，整个拍摄和采访都在虾仁和牛肉的鲜香味道中度过。他在采访前做了一件很体贴的事，也许也是最近在博客上洋洋洒洒连载《关于贝克汉姆的回忆》上了瘾，他先把我们的问题写好了答案回复过来。抱着那一叠厚厚的纸看着他在造型师的吹风机下面不停自嘲：“做艺人容易嘛！”可是他还真不像是已经做好了成为“艺人”的准备，聊天时他几乎口无遮拦什么话都讲，坦率得令人大开眼界。经纪人在旁边无可奈何地唉声叹气，造型师在身后憋着笑大喊，“别说了，别说了。”末了他还叹一句：“有时候真担心面前的发问者会不怀好意。”唉，他怎么敢肯定现在的这个不是？黄健翔在今天的镜头下看起来还是有不变的朝气，不时地搂着摄影师大笑，要求换首热情有节奏感的音乐以免睡着了，一会儿又揉着肚子懊悔吃得太多……这种可爱又稀少的赤子之心多可贵，想起来他拍的那个著名的第一支广告，一个不看球朋友因此开始喜欢他，因为他可以在拍摄时大声笑场。“有时候做事不会考虑很多，我属于比较感性的一个人。身心自由快乐，按照自己的理想去做事。当然，生活不是全都按照个人意愿来进行的，我们都要适应环境。从来都是事情改变人，人改变不了事情。”感性是千真万确的，可这个看上去永远斗志昂扬的男人并不像会为了什么而改变。在有些人眼里亲切得像熟识已久的兄弟，在另外一些人眼里却针锋相对毫不留情。冰火两重天的形容词透着生猛。可是当看到他在镜头前认真又自得其乐的摆着姿势，讲述着人人都爱上体育课的小时候那份快乐，就觉得，碰过壁之后的豪放率真，真是一份应该保护的珍宝。亲友们对他的评价是：仗义疏财、才华横溢、情商低下。放在古时候，这三个词砌在一起就该是个旷达豪爽侠士范儿了。很多人注意到他客串玩票、演戏唱歌，没发现他已经建过三所希望小学；讨论过他赚多少钱，没看到他搬了一大箱子书跑到寒冷的黑龙江，给小学生们上体育课，领着孩子们踢足球。“以后我可以心安理得地被人叫老师了。”他摸着被冻伤的嘴角十分得意。临走前，我拿着录音笔让他录点什么，他特意叫人把摄影棚的音乐声关小，然后极其入戏地大喝出了那句：“你不是一个人在战斗！”摄影棚里穿梭来往的人们忽然间就安静下来，几秒钟之后哄堂大笑。看，一个创造经典的人就是这样令人快乐。体育已经成为我的一部分，是我身上的烙印。最近一次见到黄健翔应该是在《大灌篮》里面，恰到好处的出场引得电影院里笑声连连。曾经，他是狂放不羁的。虽然经常被提醒不要在外形上过于修饰自己，谁也否认不了他在麦可风后锋芒外露却又深入人心的才华。走了很长一段路之后，他即将重新回到欧洲杯的解说台前。谁都说不好这四年的喜怒哀乐给他带来的是怎样的变化。可以肯定的是，他比从前平和了，“每一天都有不同的幸福，包括战胜烦恼和困难。”私底下，我们还是希望他还是那个“像男人那样去战斗”的恃才傲物的黄健翔，毕竟，足球是激情幻化的理想。Ｑ：知道你即将重新回来主持欧洲杯了，我们确实很兴奋，你觉得可以说这是一种“复出”吗？这次合作意味着什么？Ａ：也可以算是“复出”，因为，毕竟有一年多没有在比较好的电视平台上解说比赛了。我很感谢球迷对我的支持和怀念。这次合作，对我来说，最高兴的是看到了一个可以报答球迷的好平台，可以“务正业”的地方。Ｑ：接下来大家都盼着你的解说、你的声音能够重新在耳边响起，你自己会不会担心效果？观众没有以前的热度高之类的。Ａ：我既不担心，又有些担心。不担心是因为，一个解说员在球迷心目中的地位也好，口碑也好，绝不是一两场比赛或者一次大赛就可以形成或者改变的。正是因为千百万广大的真正球迷对我的宽容和支持，他们强大的声音给了我后来的机会。说担心是因为，前面说过的体育解说的“先天不足”，总是可以给一些别有用心的人轻易地利用，找到一些缺点和问题，更别说他们用显微镜、放大镜来找茬儿挑刺了。越是用心投入这一行的人，就越容易在这样的事情上受伤。那些根本不用心，混饭吃的解说，反而根本不会在乎这样的言论，因为早就看透了想开了麻木了。Ｑ：你会维系一个怎样的工作状态呢？Ａ：体育比赛，特别是足球比赛，本身就是充满偶然、突发元素的，这正是它的魅力所在。我能做的，是保持良好平静的心态，用职业的态度去完成工作。　我的解说并非尽善尽美，解说精彩与否离不开比赛的精彩程度，包括比赛过程中的曲折性、戏曲性、偶然性等等因素，不是我一个人能左右的。我既不需要靠一次解说去证明什么，也不需要跟什么人比较或者较劲。我在体育解说这个行业所做的一切，不是我自己可以随便总结的，也不是什么人可以作终极审判的，这个权利，只在千百万球迷和观众那里',
        
        '奥运明星写真集锦曝光　展现健康时尚（图） 来源：人民网奥运明星奥运明星大满贯――属于最强者的时刻总有那么一些人，他们的人生注定要与他人不同，因为天赋的才能和力量。他们生活的是一个既要斗智又要斗力的世界，他们展示给众人的是力与美完美结合的影像，',
        
        
        '央视紧急停播动画片《虹猫蓝兔七侠传》事件经报道后，引发了数十万网民的热烈大辩论。截止到今天早上，约有２０％的网民认为《虹猫蓝兔七侠传》内容低级，充满暴力、情色、脏口会对孩子产生严重的不良影响应该停播，而大约８０％的网民则认为该片主题是要孩子懂得真爱团结勇敢，孩子非常爱看，绝对不该停播！']

In [69]:
test=pd.DataFrame(sample)

test.columns=['文章']

In [70]:
test

,文章
0,黄健翔拍时装大片 承认口无遮拦 ２００６年之前，他只是最好的体育节目主持人之一。２００４年之...
1,奥运明星写真集锦曝光 展现健康时尚（图） 来源：人民网奥运明星奥运明星大满贯――属于最强者的...
2,央视紧急停播动画片《虹猫蓝兔七侠传》事件经报道后，引发了数十万网民的热烈大辩论。截止到今天早...


## 将导入的文章进行分词和向量化

In [71]:
#加载停用词
stopword_list = [k.strip() for k in open('stopwords.txt', encoding='utf-8') if k.strip() != '']
cutWords_list = []
 
#分词
i = 0
startTime = time.time()
for article in test['文章']:
    cutWords = [k for k in jieba.cut(article) if k not in stopword_list]
    i += 1
    if i % 1000 == 0:
        print('前%d篇文章分词共花费%.2f秒' % (i, time.time() - startTime))
    cutWords_list.append(cutWords)

In [72]:
#进行词向量编码
from tqdm import tqdm
def wordDecode(data):
    sentences =data
    emb_size = 128
    model = Word2Vec.load("word2vec_model.w2v")
    emb_matrix = []
    for seq in tqdm(sentences):
        vec = []
        for w in seq:
            if w in model.wv.key_to_index:
                vec.append(model.wv[w])
        if len(vec) > 0:
            emb_matrix.append(np.mean(vec, axis=0))
        else:
            emb_matrix.append([0] * emb_size)
    emb_matrix = np.array(emb_matrix)

    return emb_matrix

In [73]:
test=wordDecode( cutWords_list)

100%|██████████| 3/3 [00:00<00:00, 601.79it/s]


In [74]:
result=[]

In [75]:
#加载训练好的模型
for fold_ in range(5):
    print("fold n°{}".format(fold_+1))
#     加载mlp模型
    clf_mlp=joblib.load(f'mlp{fold_}.model')
    of_mlp=clf_mlp.predict(test)
    oof_mlp=np.array(of_mlp).reshape(-1,1)
    result.append(oof_mlp)
#     加载lgb模型
    clf_lgb=lgb.Booster(model_file=f'lgb_model{fold_}.txt')

    of=clf_lgb.predict(test)
    oof_lgb = np.argmax(of,1).reshape(-1,1)
    result.append(oof_lgb)

fold n°1
fold n°2
fold n°3
fold n°4
fold n°5


In [76]:
#稍作处理，返回结果
result=pd.DataFrame(np.concatenate(result,axis=1))

final_result=list(result.apply(lambda x :x.mode()[0],axis=1))

di={'0':'体育','1':'健康','2':'女人','3':'娱乐','4':'房地产','5':'教育','6':'文化','7':'新闻','8':'旅游','9':'汽车','10':'科技','11':'财经'}

final_result=[ di[str(a)] for a in final_result]


f_dic={}
for d in di.values():
    f_dic[d]=final_result.count(d)

# f_dic就是最终结果

In [77]:
f_dic

{'体育': 0,
 '健康': 0,
 '女人': 1,
 '娱乐': 1,
 '房地产': 0,
 '教育': 0,
 '文化': 1,
 '新闻': 0,
 '旅游': 0,
 '汽车': 0,
 '科技': 0,
 '财经': 0}